In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
from datetime import date, timedelta
import os
import pandas as pd

In [2]:
def download_and_append_data(url, file_path): 
    # Effettua la richiesta HTTP per ottenere il contenuto della pagina
    response = requests.get(url)
    
    # Verifica se la richiesta è stata eseguita con successo (status code 200)
    if response.status_code == 200:
        # Utilizza BeautifulSoup per analizzare l'HTML della pagina
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Trova la tabella desiderata nel documento HTML
        table = soup.find_all('table')[1]
        
        # Estrai i dati dalla tabella
        data = []
        for row in table.find_all('tr'):
            columns = row.find_all(['td', 'th'])[:9]
            data.append([column.get_text(strip=True) for column in columns] + [pd.to_datetime('now')])

        # Se il file esiste, apri il file CSV e aggiungi i nuovi dati
        if os.path.exists(file_path):
            try:
                existing_data = pd.read_csv(file_path)
            except pd.errors.EmptyDataError:
                # Se il file è vuoto, crea un DataFrame vuoto
                existing_data = pd.DataFrame()

            new_data = pd.DataFrame(data, columns=existing_data.columns)
            updated_data = pd.concat([existing_data, new_data], ignore_index=False)
            updated_data.to_csv(file_path, index=False)
        else:
            # Se il file non esiste, crea un nuovo file CSV
            with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerows(data)

        print(f'I dati sono stati scaricati con successo e aggiunti a {file_path}')
    else:
        print(f'Errore nella richiesta HTTP. Codice di stato: {response.status_code}')

# URL della pagina con la tabella
page_url = 'https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ=='


# Nome del file CSV
file_name = 'cinetel_.csv'
file_path = os.path.join(os.getcwd(), file_name)

In [3]:
download_and_append_data(page_url, file_path)

I dati sono stati scaricati con successo e aggiunti a e:\data_analysis_python\cinetel\cinetel_.csv
